In [2]:
#  dcmpandas installed directly from github with ....
#  pip install -e git+https://github.com/dov/dcmpandas#egg=dcmpandas
# import dcmpandas as dp
# dp.scrape('.')


#import matplotlib.pyplot as plt

# pip install pydicom
# pip install Pillow -U

import numpy as np

import pydicom 
from  PIL import Image


In [89]:
dicom = pydicom.dcmread('./Pantex/DICOM/PAT_0000/STD_0001/SER_0000/OBJ_0001/IM_0001')
print(f"{dicom.PatientID} sex:{dicom.PatientSex} age:{dicom.PatientBirthDate}")

# print(dicom.fileobj_type, dicom.file_meta)

print(f"\n{dicom.get_item([0x0008, 0x1030])}\n") 
# print( dicom.file_meta)
# for i in iter( dicom.iterall()):
#   print(i)
dicom_dict = dicom.to_json_dict()
print(dicom_dict['00181030']['Value'][0])
# /Users/white/Documents/AlexProjects/DICOM_OCC_MED/Pantex/DICOM/PAT_0000/STD_0001/SER_0000/OBJ_0001
# dicom = pydicom.dcmread('./dicom/ILO_Standards/4EF026B7') #read the dicom file into a variable
image_pixel_array = dicom.pixel_array.astype(float) # extract the image pixel data from the dicom dictionary
rescaled_image_pixel_array = (np.maximum(image_pixel_array,0)/image_pixel_array.max())*255 # float pixels
final_image_pixel_array = np.uint8(rescaled_image_pixel_array) #convert to integer pixels
finale_image = Image.fromarray(final_image_pixel_array) #use image library to convert array of pixel data to an image
finale_image.show() # will open a copy of the image in Preview
# finale_image.save('CXR.PNG') #  Saves to file



P05-0181 sex:M age:19330313

RawDataElement(tag=(0008, 1030), VR='LO', length=12, value=b'CHEST 1 VIEW', value_tell=700, is_implicit_VR=False, is_little_endian=True, is_raw=True)

Chest AP


In [99]:
import glob
study_types = [] 
for f in glob.glob('./Pantex/**/IM*', recursive=True):
  dicom = pydicom.dcmread(f)
  study_description = str( dicom.get_item([0x0008, 0x1030]).value)
  #Find is the image is an X-ray or a different type of image (like a scanned ILO form)
  image_type = "NOT X-RAY"
  try:
    dicom_dict = dicom.to_json_dict()
    image_type=dicom_dict['00181030']['Value'][0] # Will have value like 'CHEST PA' or 'CHEST LAT', we are only interested in AP and PA films
    study_types.append(image_type)
  except Exception:
    pass
list(set( study_types))



['CHEST', 'Chest LAT', 'Chest PA', 'Chest AP']

In [100]:
import glob

for f in glob.glob('./Pantex/**/IM*', recursive=True):
  dicom = pydicom.dcmread(f)
  study_description = str( dicom.get_item([0x0008, 0x1030]).value)
  #Find is the image is an X-ray or a different type of image (like a scanned ILO form)
  image_type = "NOT X-RAY"
  try:
    dicom_dict = dicom.to_json_dict()
    image_type=dicom_dict['00181030']['Value'][0] # Will have value like 'CHEST PA' or 'CHEST LAT', we are only interested in AP and PA films
  except Exception:
    pass
  if((image_type !='Chest LAT') and (image_type !='NOT X-RAY') ):
    # ANONYMIZE, JUST IN CASE (should already be stripped)
    dicom.PatientName = None # 'ANON'
    dicom.PatientBirthDate = None #'19000101'
    PA_or_AP = 'AP' if image_type =='Chest AP' else 'PA'
    filename = f"{dicom.PatientID}_{dicom.StudyDate}_{PA_or_AP}.dcm"
    print(f,filename)
    dicom.save_as("Pantex/FLATTENED/"+filename)
    # files.append([f,study_description,image_type])
    # image_pixel_array = dicom.pixel_array.astype(float) # extract the image pixel data from the dicom dictionary
    # rescaled_image_pixel_array = (np.maximum(image_pixel_array,0)/image_pixel_array.max())*255 # float pixels
    # final_image_pixel_array = np.uint8(rescaled_image_pixel_array) #convert to integer pixels
    # finale_image = Image.fromarray(final_image_pixel_array) #use image library to convert array of pixel data to an image
    # finale_image.show()
# len(files)

  

./Pantex/DICOM/PAT_0011/STD_0001/SER_0000/OBJ_0001/IM_0001 P13-0043_20131010_AP.dcm
./Pantex/DICOM/PAT_0016/STD_0000/SER_0000/OBJ_0001/IM_0001 P06-0172_20110901_AP.dcm
./Pantex/DICOM/PAT_0017/STD_0003/SER_0000/OBJ_0001/IM_0001 P10-0066_20130725_AP.dcm
./Pantex/DICOM/PAT_0017/STD_0002/SER_0000/OBJ_0001/IM_0001 P10-0066_20100630_PA.dcm
./Pantex/DICOM/PAT_0010/STD_0001/SER_0000/OBJ_0001/IM_0001 P13-0046_20131017_AP.dcm
./Pantex/DICOM/PAT_0010/STD_0001/SER_0001/OBJ_0001/IM_0001 P13-0046_20131017_AP.dcm
./Pantex/DICOM/PAT_0003/STD_0001/SER_0000/OBJ_0001/IM_0001 P05-0157_20150514_AP.dcm
./Pantex/DICOM/PAT_0003/STD_0002/SER_0000/OBJ_0001/IM_0004 P05-0157_20091105_PA.dcm
./Pantex/DICOM/PAT_0003/STD_0002/SER_0000/OBJ_0001/IM_0002 P05-0157_20091105_PA.dcm
./Pantex/DICOM/PAT_0004/STD_0001/SER_0000/OBJ_0001/IM_0001 P112299_20091028_PA.dcm
./Pantex/DICOM/PAT_0005/STD_0000/SER_0001/OBJ_0001/IM_0001 P09-0101_20211209_PA.dcm
./Pantex/DICOM/PAT_0002/STD_0000/SER_0000/OBJ_0001/IM_0001 XP11-0015_20110330